# Redis
* https://redis.io/docs/latest/
* https://github.com/redis/redis
* [Redis benchmark](https://redis.io/docs/latest/operate/oss_and_stack/management/optimization/benchmarks/)


> [Redis](https://redis.io/about/)
> Redis is an in-memory data store used by millions of developers as a cache, vector database, document database, streaming engine, and message broker. Redis has built-in replication and different levels of on-disk persistence. It supports complex data types (for example, strings, hashes, lists, sets, sorted sets, and JSON), with atomic operations defined on those data types.

> [Redis Stack](https://redis.io/about/about-stack/)
> Redis Stack extends the core capabilities of Redis OSS and provides a complete developer experience for debugging and more.
> 
> Redis Stack is the best starting point for working with Redis. We’ve bundled together the best of the technology that we have to offer into an easy-to-use package.
> 
> In addition to all of the features of Redis OSS, Redis Stack supports:
>
>- Pobabilistic data structures
>- Queryable JSON documents
>- Querying across hashes and JSON documents
>- Time series data support (ingestion & querying), including full-text search
>- Graph data models with the Cypher query language

Misc:
- [Redis.md](./Redis.md)
- Redis设计与实现.
- [Redis进阶 - 事务：Redis事务详解](https://pdai.tech/md/db/nosql-redis/db-redis-x-trans.html)
- [Limiting simultaneous tasks using distributed semaphores](https://medium.com/picus-security-engineering/limiting-simultaneous-tasks-using-distributed-semaphores-0f7aab47395a) - 2023-12-06
- [Redis Explained](https://architecturenotes.co/redis/) - 2022-08-11

actions:
- https://github.com/jargonzhou/application-store/tree/main/data-engineering/redis
  - [How do you perform a HEALTHCHECK in the Redis Docker image?](https://stackoverflow.com/questions/67904609/how-do-you-perform-a-healthcheck-in-the-redis-docker-image)


# Doc

- [Develop with Redis](https://redis.io/docs/latest/develop/)
	- Connect: CLI, Redis Insight, Clients
	- Understand Redis data types: - 数据结构
		- Strings, Lists, Sets, Hashes, Sorted sets, Streams
		- Geospatial indexes, Bitmaps, Bitfields, HyperLogLog
	- Interact with data in Redis
		- Search and query
		- Programmability: Functions, Lua Scripting
		- Transactions: `MULTI`, `EXEC`, `DISCARD`, `WATCH`
		- Pub/sub: `SUBSCRIBE`, `UNSUBSCRIBE`, `PUBLISH`
	- Use Redis
		- Keyspace
		- Client-side caching
		- Pipelining
		- Keyspace notifications
		- Patterns
	- Reference
		- Command tips
		- Modules API
		- Sentinel clients
		- Command key specifications
		- Protocol spec
		- Client handling
		- Eviction
		- Command arguments
		- Gopher protocol
- [Libraries and tools](https://redis.io/docs/latest/integrate/)
- [Commands](https://redis.io/docs/latest/commands/)
- [APIs](https://redis.io/docs/latest/apis/)
	- APIs for Developers
		- Client API: 
			- [Redis commands](https://redis.io/docs/latest/commands/)
			- [Connect with Redis clients introduction](https://redis.io/docs/latest/develop/connect/clients/)
		- Programmability APIs
			- Lua scripts
				- [Redis Lua API reference](https://redis.io/docs/latest/develop/interact/programmability/lua-api/)
				- [Scripting with Lua introduction](https://redis.io/docs/latest/develop/interact/programmability/eval-intro/)
			- Functions: in Redis 7, supersedes the use of Lua
				- [Functions and scripting in Redis 7 and beyond](https://redis.io/docs/latest/develop/interact/programmability/functions-intro/)
			- Modules API: new commands
			- [Redis Modules API introduction](https://redis.io/docs/latest/develop/reference/modules/)
			- [Redis Modules API reference](https://redis.io/docs/latest/develop/reference/modules/modules-api-ref/)
	- APIs for Operators
		- Redis Cloud API, Redis Enterprise Software API, Redis Enterprise for Kubernetes API
- Products
- Redis Cloud
- Redis Enterprise Software
- Redis Enterprise for Kubernetes
- [Redis Community Edition and Stack](https://redis.io/docs/latest/operate/oss_and_stack/)
	- [Redis cluster specification](https://redis.io/docs/latest/operate/oss_and_stack/reference/cluster-spec/)
	- [Redis internals](https://redis.io/docs/latest/operate/oss_and_stack/reference/internals/)
- Redis Insight 

# Concepts

## Key eviction
- https://redis.io/docs/reference/eviction/


The following policies are available:
- `noeviction`: New values aren’t saved when memory limit is reached. When a database uses replication, this applies to the primary database
- `allkeys-lru`: Keeps most recently used keys; removes least recently used (LRU) keys
- `allkeys-lfu`: Keeps frequently used keys; removes least frequently used (LFU) keys
- `volatile-lru`: Removes least recently used keys with the expire field set to true.
- `volatile-lfu`: Removes least frequently used keys with the expire field set to true.
- `allkeys-random`: Randomly removes keys to make space for the new data added.
- `volatile-random`: Randomly removes keys with expire field set to true.
- `volatile-ttl`: Removes keys with expire field set to true and the shortest remaining time-to-live (TTL) value.


## The Redlock Algorithm
- https://redis.io/docs/manual/patterns/distributed-locks/


In the distributed version of the algorithm we assume we have *N Redis masters*. 
Those nodes are *totally independent*, so we don’t use replication or any other implicit coordination system. 

We already described how to acquire and release the lock safely in a single instance. 
We take for granted that the algorithm will *use this method to acquire and release the lock in a single instance*. 

In our examples we set *N=5*, which is a reasonable value, so we need to run 5 Redis masters on different computers or virtual machines in order to ensure that they’ll fail in a mostly independent way.

In order to acquire the lock, the client performs the following operations:

- (1) It gets the current time in milliseconds.
- (2) It tries to acquire the lock in all the N instances sequentially, using the *same key name and random value* in all the instances. During step 2, when setting the lock in each instance, the client uses a *timeout* which is small compared to the total lock auto-release time in order to acquire it. For example if the auto-release time is 10 seconds, the timeout could be in the ~ 5-50 milliseconds range. This prevents the client from remaining blocked for a long time trying to talk with a Redis node which is down: if an instance is not available, we should try to talk with the next instance ASAP.
- (3) The client computes how much time elapsed in order to acquire the lock, by *subtracting* from the current time the timestamp obtained in step 1. If and only if the client was able to acquire the lock in the majority of the instances (at least 3), and the total time elapsed to acquire the lock is less than lock validity time, the lock is considered to be acquired.
- (4) If the lock was acquired, its validity time is considered to be the initial validity time minus the time elapsed, as computed in step 3.
- (5) If the client failed to acquire the lock for some reason (either it was not able to lock N/2+1 instances or the validity time is negative), it will try to unlock all the instances (even the instances it believed it was not able to lock).

# Commands

## SET


```
SET key value [NX | XX] [GET] [EX seconds | PX milliseconds |
  EXAT unix-time-seconds | PXAT unix-time-milliseconds | KEEPTTL]
```

- EX seconds -- Set the specified expire time, in seconds.
- PX milliseconds -- Set the specified expire time, in milliseconds.
- EXAT timestamp-seconds -- Set the specified Unix time at which the key will expire, in seconds.
- PXAT timestamp-milliseconds -- Set the specified Unix time at which the key will expire, in milliseconds.
- NX -- Only set the key if it does not already exist.
- XX -- Only set the key if it already exist.
- KEEPTTL -- Retain the time to live associated with the key.
- GET -- Return the old string stored at key, or nil if key did not exist. An error is returned and SET aborted if the value stored at key is not a string.

# Clients

Java:
- [Jedis](https://github.com/redis/jedis): Redis Java client
- [Lettuce](https://github.com/redis/lettuce): Lettuce is a scalable thread-safe Redis client for synchronous, asynchronous and reactive usage. Multiple threads may share one connection if they avoid blocking and transactional operations such as BLPOP and MULTI/EXEC. Lettuce is built with netty. Supports advanced Redis features such as Sentinel, Cluster, Pipelining, Auto-Reconnect and Redis data models.
	- [Lettuce Reference Guide](https://redis.github.io/lettuce/overview/)
	- [Streaming API](https://github.com/redis/lettuce/wiki/Streaming-API): `StreamingChannel`.
- [Redisson](https://github.com/redisson/redisson): Redisson - Easy Redis Java client and Real-Time Data Platform. Valkey compatible. Sync/Async/RxJava/Reactive API. Over 50 Redis based Java objects and services: Set, Multimap, SortedSet, Map, List, Queue, Deque, Semaphore, Lock, AtomicLong, Map Reduce, Bloom filter, Spring Cache, Tomcat, Scheduler, JCache API, Hibernate, RPC, local cache ...
- [redis-mock-java](https://github.com/wilkenstein/redis-mock-java): An in-memory implementation of redis in Java. - last release: 2015-04-25

# RTFSC

version: 7.2.

Read `README.md` in code repo!!!

install extensions:
- C/C++ for Visual Studio Code
- VS Code Makefile Tools
	- Lanuch target: src/redis-server
	- Makefile: Debug the selected binary target

## src


Use 'VS Code Counter' to gain the code line counts:

| File                                      | Line               | Description                                                                                                                                                                                                                                                                                                  |
| :---------------------------------------- | :----------------- | :----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| Makefile                                  | 515                |                                                                                                                                                                                                                                                                                                              |
| acl.c                                     | 3151               | ACL                                                                                                                                                                                                                                                                                                          |
| adlist.c, adlist.h                        | 418, 101           | Linked lists                                                                                                                                                                                                                                                                                                 |
| ae.c, ae.h                                | 513, 137           | A simple event-driven programming library.                                                                                                                                                                                                                                                                   |
| ae_epoll.c                                | 140                | Linux epoll(2) based ae.c module                                                                                                                                                                                                                                                                             |
| ae_evport.c                               | 322                | ae.c module for illumos event ports.                                                                                                                                                                                                                                                                         |
| ae_kqueue.c                               | 191                | Kqueue(2)-based ae.c module                                                                                                                                                                                                                                                                                  |
| ae_select.c                               | 111                | Select()-based ae.c module.                                                                                                                                                                                                                                                                                  |
| anet.c, anet.h                            | 707, 76            | Networking the easy way                                                                                                                                                                                                                                                                                      |
| aof.c                                     | 2745               | AOF: Append-only files                                                                                                                                                                                                                                                                                       |
| asciilogo.h                               | 48                 | ascii_logo                                                                                                                                                                                                                                                                                                   |
| atomicvar.h                               | 159                | implements atomic counters using c11 `_Atomic`, `__atomic` or `__sync` macros if available, otherwise we will throw an error when compile.                                                                                                                                                                   |
| bio.c, bio.h                              | 346, 55            | Background I/O service for Redis.                                                                                                                                                                                                                                                                            |
| bitops.c                                  | 1268               | 数据类型 Bitfields                                                                                                                                                                                                                                                                                               |
| blocked.c                                 | 768                | generic support for blocking operations like `BLPOP` & `WAIT`.                                                                                                                                                                                                                                               |
| call_reply.c, call_reply.h                | 561, 61            | parse a `RESP` protocol reply and represent it                                                                                                                                                                                                                                                               |
| childinfo.c                               | 184                | a child-parent channel used in order to move information about the RDB / AOF saving process from the child to the parent                                                                                                                                                                                     |
| cli_commands.c, cli_commands.h            | 14, 47             | redis-cli                                                                                                                                                                                                                                                                                                    |
| cli_common.c, cli_common.h                | 409, 55            | CLI (command line interface) common methods                                                                                                                                                                                                                                                                  |
| cluster.c, cluster.h                      | 7721, 448          | Cluster implementation.                                                                                                                                                                                                                                                                                      |
| commands.c, commands.h                    | 14, 41             | `redisCommandTable`                                                                                                                                                                                                                                                                                          |
| config.c, config.h                        | 3414, 322          | Configuration file parsing and `CONFIG` `GET`/`SET` commands implementation.                                                                                                                                                                                                                                 |
| connection.c, connection.h                | 209, 455           | Connection abstraction                                                                                                                                                                                                                                                                                       |
| connhelpers.h                             | 89                 | Currently helpers implement the mechanisms for invoking connection handlers and tracking connection references, to allow safe destruction of connections from within a handler.                                                                                                                              |
| crc16.c                                   | 89                 | CRC16 implementation according to CCITT standards.                                                                                                                                                                                                                                                           |
| crc16_slottable.h                         | 836                | `crc16_slot_table`                                                                                                                                                                                                                                                                                           |
| crc64.c, crc64.h                          | 162, 14            | [pycrc](https://pycrc.org/) generated Cyclic Redundancy Check (CRC) C source code.                                                                                                                                                                                                                           |
| crcspeed.c, crcspeed.h                    | 283, 61            |                                                                                                                                                                                                                                                                                                              |
| db.c                                      | 2559               | C-level DB API                                                                                                                                                                                                                                                                                               |
| debug.c                                   | 2323               | Debugging                                                                                                                                                                                                                                                                                                    |
| debugmacro.h                              | 47                 | debugging macros to be used when investigating issues.                                                                                                                                                                                                                                                       |
| defrag.c                                  | 1081               | Active memory defragmentation                                                                                                                                                                                                                                                                                |
| dict.c, dict.h                            | 1750, 232          | Hash Tables Implementation.                                                                                                                                                                                                                                                                                  |
| endianconv.c, endianconv.h                | 130, 79            | Endian conversions utilities.                                                                                                                                                                                                                                                                                |
| eval.c                                    | 1668               | `EVAL` and `SCRIPT` commands implementation. LDB: Redis Lua debugging facilities                                                                                                                                                                                                                             |
| evict.c                                   | 758                | Maxmemory directive handling (LRU eviction and other policies).                                                                                                                                                                                                                                              |
| expire.c                                  | 755                | Implementation of `EXPIRE` (keys with fixed time to live).                                                                                                                                                                                                                                                   |
| fmacros.h                                 | 77                 | 文件宏                                                                                                                                                                                                                                                                                                          |
| function_lua.c                            | 507                | provides the Lua engine functionality                                                                                                                                                                                                                                                                        |
| functions.c, functions.h                  | 1140, 137          | Functions API                                                                                                                                                                                                                                                                                                |
| geo.c, geo.h                              | 1006, 23           | 数据类型 Geospatial indexes. `GEOADD`, `GEORADIUS`, `GEORADIUSBYMEMBER`, `GEODIST`, `GEOPOS`, `GEOSEARCH`, `GEOSEARCHSTORE`.                                                                                                                                                                                     |
| geohash.c, geohash.h                      | 300, 136           | `GEOHASH`                                                                                                                                                                                                                                                                                                    |
| geohash_helper.c, geohash_helper.h        | 281, 66            |                                                                                                                                                                                                                                                                                                              |
| hyperloglog.c                             | 1619               | 数据类型 HyperLogLog                                                                                                                                                                                                                                                                                             |
| intset.c, intset.h                        | 561, 58            | `intset`                                                                                                                                                                                                                                                                                                     |
| latency.c, latency.h                      | 740, 109           | Latency monitor API                                                                                                                                                                                                                                                                                          |
| lazyfree.c                                | 233                |                                                                                                                                                                                                                                                                                                              |
| listpack.c, listpack.h, listpack_malloc.h | 2661, 107, 50      | A lists of strings serialization format                                                                                                                                                                                                                                                                      |
| localtime.c                               | 124                | a safe version of localtime() which contains no locks and is `fork()` friendly.                                                                                                                                                                                                                              |
| logreqres.c                               | 316                | implements the interface of logging clients' requests and responses into a file                                                                                                                                                                                                                              |
| lolwut.c, lolwut.h, lolwut5.c, lolwut6.c  | 189, 56, 178, 202  | `LOLWUT` Displays computer art and the Redis version                                                                                                                                                                                                                                                         |
| lzf_c.c, lzf_d.c, lzfP.h, lzf.h           | 303, 192, 191, 101 | lzf -- an extremely fast/free compression/decompression-method                                                                                                                                                                                                                                               |
| memtest.c                                 | 378                | RESTART HERE!!!                                                                                                                                                                                                                                                                                              |
| mkreleasehdr.sh                           | 17                 |                                                                                                                                                                                                                                                                                                              |
| module.c                                  | 13848              |                                                                                                                                                                                                                                                                                                              |
| monotonic.c, monotonic.h                  | 181, 62            |                                                                                                                                                                                                                                                                                                              |
| mt19937-64.c, mt19937-64.h                | 188, 88            |                                                                                                                                                                                                                                                                                                              |
| multi.c                                   | 501                |                                                                                                                                                                                                                                                                                                              |
| networking.c                              | 4579               |                                                                                                                                                                                                                                                                                                              |
| notify.c                                  | 146                |                                                                                                                                                                                                                                                                                                              |
| object.c                                  | 1689               |                                                                                                                                                                                                                                                                                                              |
| pqsort.c, pqsort.h                        | 186, 41            |                                                                                                                                                                                                                                                                                                              |
| pubsub.c                                  | 755                | Pub/Sub                                                                                                                                                                                                                                                                                                      |
| quicklist.c, quicklist.h                  | 3304, 215          | Lists are encoded as linked lists of N-elements flat arrays: A doubly linked list of listpacks.                                                                                                                                                                                                              |
| rand.c, rand.h                            | 94, 39             |                                                                                                                                                                                                                                                                                                              |
| rax.c, rax.h, rax_malloc.h                | 1928, 217, 45      | A radix tree implementation.                                                                                                                                                                                                                                                                                 |
| rdb.c, rdb.h                              | 3723, 182          | RDB                                                                                                                                                                                                                                                                                                          |
| redis-benchmark.c                         | 2061               |                                                                                                                                                                                                                                                                                                              |
| redis-check-aof.c                         | 575                |                                                                                                                                                                                                                                                                                                              |
| redis-check-rdb.c                         | 450                |                                                                                                                                                                                                                                                                                                              |
| redis-cli.c                               | 9985               |                                                                                                                                                                                                                                                                                                              |
| redis-trib.rb                             | 130                |                                                                                                                                                                                                                                                                                                              |
| redisassert.c, redisassert.h              | 54, 50             |                                                                                                                                                                                                                                                                                                              |
| redismodule.h                             | 1686               | Redis modules API defines.                                                                                                                                                                                                                                                                                   |
| release.c                                 | 70                 |                                                                                                                                                                                                                                                                                                              |
| replication.c                             | 4242               |                                                                                                                                                                                                                                                                                                              |
| resp_parser.c, resp_parser.h              | 229, 95            | A RESP parser for parsing replies returned by `RM_Call` or Lua's `redis.call()`.                                                                                                                                                                                                                             |
| rio.c, rio.h                              | 521, 186           | a simple stream-oriented I/O abstraction that provides an interface to write code that can consume/produce data using different concrete input and output devices. For instance the same rdb.c code using the rio abstraction can be used to read and write the RDB format using in-memory buffers or files. |
| script.c, script.h                        | 579, 112           |                                                                                                                                                                                                                                                                                                              |
| script_lua.c, script_lua.h                | 1732, 88           |                                                                                                                                                                                                                                                                                                              |
| sds.c, sds.h, sdsalloc.h                  | 1497, 288, 55      | SDSLib 2.0 -- A C dynamic strings library                                                                                                                                                                                                                                                                    |
| sentinel.c                                | 5485               | Sentinel implementation                                                                                                                                                                                                                                                                                      |
| server.c, server.h                        | 7374, 3746         | Server                                                                                                                                                                                                                                                                                                       |
| setcpuaffinity.c                          | 156                |                                                                                                                                                                                                                                                                                                              |
| setproctitle.c                            | 332                |                                                                                                                                                                                                                                                                                                              |
| sha1.c, sha1.h                            | 240, 28            |                                                                                                                                                                                                                                                                                                              |
| sha256.c, sha256.h                        | 164, 36            |                                                                                                                                                                                                                                                                                                              |
| siphash.c                                 | 374                |                                                                                                                                                                                                                                                                                                              |
| slowlog.c, slowlog.h                      | 207, 52            |                                                                                                                                                                                                                                                                                                              |
| socket.c                                  | 472                |                                                                                                                                                                                                                                                                                                              |
| solarisfixes.h                            | 55                 |                                                                                                                                                                                                                                                                                                              |
| sort.c                                    | 620                |                                                                                                                                                                                                                                                                                                              |
| sparkline.c, sparkline.h                  | 180, 57            | ASCII graphs API                                                                                                                                                                                                                                                                                             |
| stream.h                                  | 148                |                                                                                                                                                                                                                                                                                                              |
| strl.c                                    | 87                 |                                                                                                                                                                                                                                                                                                              |
| syncio.c                                  | 146                |                                                                                                                                                                                                                                                                                                              |
| syscheck.c, syscheck.h                    | 376, 47            |                                                                                                                                                                                                                                                                                                              |
| t_hash.c                                  | 1164               | 数据类型 Hashes                                                                                                                                                                                                                                                                                                  |
| t_list.c                                  | 1389               | 数据类型 List                                                                                                                                                                                                                                                                                                    |
| t_set.c                                   | 1681               | 数据类型 Set                                                                                                                                                                                                                                                                                                     |
| t_stream.c                                | 4039               | 数据类型 Stream                                                                                                                                                                                                                                                                                                  |
| t_string.c                                | 952                | 数据类型 String                                                                                                                                                                                                                                                                                                  |
| t_zset.c                                  | 4462               | 数据类型 Sorted set                                                                                                                                                                                                                                                                                              |
| testhelp.h                                | 63                 |                                                                                                                                                                                                                                                                                                              |
| timeout.c                                 | 203                |                                                                                                                                                                                                                                                                                                              |
| tls.c                                     | 1205               |                                                                                                                                                                                                                                                                                                              |
| tracking.c                                | 661                |                                                                                                                                                                                                                                                                                                              |
| unix.c                                    | 208                |                                                                                                                                                                                                                                                                                                              |
| util.c, util.h                            | 1432, 100          | Misc functions useful in many places                                                                                                                                                                                                                                                                         |
| version.h                                 | 3                  | Version macro                                                                                                                                                                                                                                                                                                |
| ziplist.c, ziplist.h                      | 2667, 75           | Compact list data structure                                                                                                                                                                                                                                                                                  |
| zipmap.c, zipmap.h                        | 543, 55            | String -> String Map data structure optimized for size.                                                                                                                                                                                                                                                      |
| zmalloc.c, zmalloc.h                      | 852, 168           | total memory usage aware version of `malloc`/`free`                                                                                                                                                                                                                                                          |



## modules

- [modules API](https://redis.io/docs/latest/develop/reference/modules/)
- [community-supported modules](https://redis.io/docs/latest/operate/oss_and_stack/stack-with-enterprise/)

| File           | Line | Description |
| :------------- | :--- | :---------- |
| Makefile       | 70   |             |
| helloacl.c     | 191  |             |
| helloblock.c   | 219  |             |
| hellocluster.c | 119  |             |
| hellodict.c    | 132  |             |
| hellohook.c    | 93   |             |
| hellotimer.c   | 76   |             |
| hellotype.c    | 363  |             |
| helloworld.c   | 622  |             |


## commands


| File                                  | Line | Description |
| :------------------------------------ | :--- | :---------- |
| acl-cat.json                          | 43   |             |
| acl-deluser.json                      | 34   |             |
| acl-dryrun.json                       | 48   |             |
| acl-genpass.json                      | 29   |             |
| acl-getuser.json                      | 92   |             |
| acl-help.json                         | 24   |             |
| acl-list.json                         | 26   |             |
| acl-load.json                         | 22   |             |
| acl-log.json                          | 91   |             |
| acl-save.json                         | 26   |             |
| acl-setuser.json                      | 48   |             |
| acl-users.json                        | 26   |             |
| acl-whoami.json                       | 22   |             |
| acl.json                              | 13   |             |
| append.json                           | 54   |             |
| asking.json                           | 20   |             |
| auth.json                             | 44   |             |
| bgrewriteaof.json                     | 20   |             |
| bgsave.json                           | 41   |             |
| bitcount.json                         | 88   |             |
| bitfield.json                         | 160  |             |
| bitfield_ro.json                      | 70   |             |
| bitop.json                            | 100  |             |
| bitpos.json                           | 107  |             |
| blmove.json                           | 118  |             |
| blmpop.json                           | 106  |             |
| blpop.json                            | 81   |             |
| brpop.json                            | 80   |             |
| brpoplpush.json                       | 97   |             |
| bzmpop.json                           | 118  |             |
| bzpopmax.json                         | 86   |             |
| bzpopmin.json                         | 86   |             |
| client-caching.json                   | 42   |             |
| client-getname.json                   | 33   |             |
| client-getredir.json                  | 38   |             |
| client-help.json                      | 27   |             |
| client-id.json                        | 25   |             |
| client-info.json                      | 28   |             |
| client-kill.json                      | 160  |             |
| client-list.json                      | 94   |             |
| client-no-evict.json                  | 43   |             |
| client-no-touch.json                  | 41   |             |
| client-pause.json                     | 55   |             |
| client-reply.json                     | 48   |             |
| client-setinfo.json                   | 46   |             |
| client-setname.json                   | 34   |             |
| client-tracking.json                  | 81   |             |
| client-trackinginfo.json              | 81   |             |
| client-unblock.json                   | 57   |             |
| client-unpause.json                   | 25   |             |
| client.json                           | 13   |             |
| cluster-addslots.json                 | 27   |             |
| cluster-addslotsrange.json            | 37   |             |
| cluster-bumpepoch.json                | 34   |             |
| cluster-count-failure-reports.json    | 30   |             |
| cluster-countkeysinslot.json          | 26   |             |
| cluster-delslots.json                 | 27   |             |
| cluster-delslotsrange.json            | 37   |             |
| cluster-failover.json                 | 39   |             |
| cluster-flushslots.json               | 20   |             |
| cluster-forget.json                   | 26   |             |
| cluster-getkeysinslot.json            | 36   |             |
| cluster-help.json                     | 23   |             |
| cluster-info.json                     | 22   |             |
| cluster-keyslot.json                  | 26   |             |
| cluster-links.json                    | 61   |             |
| cluster-meet.json                     | 42   |             |
| cluster-myid.json                     | 19   |             |
| cluster-myshardid.json                | 23   |             |
| cluster-nodes.json                    | 22   |             |
| cluster-replicas.json                 | 33   |             |
| cluster-replicate.json                | 26   |             |
| cluster-reset.json                    | 39   |             |
| cluster-saveconfig.json               | 20   |             |
| cluster-set-config-epoch.json         | 26   |             |
| cluster-setslot.json                  | 55   |             |
| cluster-shards.json                   | 91   |             |
| cluster-slaves.json                   | 38   |             |
| cluster-slots.json                    | 137  |             |
| cluster.json                          | 10   |             |
| command-count.json                    | 24   |             |
| command-docs.json                     | 212  |             |
| command-getkeys.json                  | 40   |             |
| command-getkeysandflags.json          | 56   |             |
| command-help.json                     | 27   |             |
| command-info.json                     | 214  |             |
| command-list.json                     | 56   |             |
| command.json                          | 22   |             |
| config-get.json                       | 37   |             |
| config-help.json                      | 23   |             |
| config-resetstat.json                 | 25   |             |
| config-rewrite.json                   | 25   |             |
| config-set.json                       | 48   |             |
| config.json                           | 10   |             |
| copy.json                             | 92   |             |
| dbsize.json                           | 26   |             |
| debug.json                            | 21   |             |
| decr.json                             | 51   |             |
| decrby.json                           | 55   |             |
| del.json                              | 54   |             |
| discard.json                          | 24   |             |
| dump.json                             | 59   |             |
| echo.json                             | 29   |             |
| eval.json                             | 70   |             |
| eval_ro.json                          | 69   |             |
| evalsha.json                          | 69   |             |
| evalsha_ro.json                       | 68   |             |
| exec.json                             | 32   |             |
| exists.json                           | 59   |             |
| expire.json                           | 95   |             |
| expireat.json                         | 95   |             |
| expiretime.json                       | 62   |             |
| failover.json                         | 55   |             |
| fcall.json                            | 70   |             |
| fcall_ro.json                         | 69   |             |
| flushall.json                         | 56   |             |
| flushdb.json                          | 56   |             |
| function-delete.json                  | 32   |             |
| function-dump.json                    | 22   |             |
| function-flush.json                   | 45   |             |
| function-help.json                    | 26   |             |
| function-kill.json                    | 26   |             |
| function-list.json                    | 88   |             |
| function-load.json                    | 40   |             |
| function-restore.json                 | 55   |             |
| function-stats.json                   | 82   |             |
| function.json                         | 10   |             |
| geoadd.json                           | 99   |             |
| geodist.json                          | 92   |             |
| geohash.json                          | 57   |             |
| geopos.json                           | 77   |             |
| georadius.json                        | 271  |             |
| georadius_ro.json                     | 202  |             |
| georadiusbymember.json                | 262  |             |
| georadiusbymember_ro.json             | 191  |             |
| geosearch.json                        | 268  |             |
| geosearchstore.json                   | 229  |             |
| get.json                              | 57   |             |
| getbit.json                           | 60   |             |
| getdel.json                           | 58   |             |
| getex.json                            | 91   |             |
| getrange.json                         | 56   |             |
| getset.json                           | 68   |             |
| hdel.json                             | 60   |             |
| hello.json                            | 112  |             |
| hexists.json                          | 60   |             |
| hget.json                             | 61   |             |
| hgetall.json                          | 54   |             |
| hincrby.json                          | 59   |             |
| hincrbyfloat.json                     | 59   |             |
| hkeys.json                            | 55   |             |
| hlen.json                             | 48   |             |
| hmget.json                            | 65   |             |
| hmset.json                            | 69   |             |
| hrandfield.json                       | 102  |             |
| hscan.json                            | 82   |             |
| hset.json                             | 71   |             |
| hsetnx.json                           | 66   |             |
| hstrlen.json                          | 53   |             |
| hvals.json                            | 54   |             |
| incr.json                             | 51   |             |
| incrby.json                           | 55   |             |
| incrbyfloat.json                      | 55   |             |
| info.json                             | 42   |             |
| keys.json                             | 35   |             |
| lastsave.json                         | 27   |             |
| latency-doctor.json                   | 27   |             |
| latency-graph.json                    | 33   |             |
| latency-help.json                     | 23   |             |
| latency-histogram.json                | 55   |             |
| latency-history.json                  | 50   |             |
| latency-latest.json                   | 50   |             |
| latency-reset.json                    | 34   |             |
| latency.json                          | 10   |             |
| lcs.json                              | 128  |             |
| lindex.json                           | 60   |             |
| linsert.json                          | 86   |             |
| llen.json                             | 49   |             |
| lmove.json                            | 105  |             |
| lmpop.json                            | 101  |             |
| lolwut.json                           | 26   |             |
| lpop.json                             | 78   |             |
| lpos.json                             | 86   |             |
| lpush.json                            | 61   |             |
| lpushx.json                           | 62   |             |
| lrange.json                           | 59   |             |
| lrem.json                             | 57   |             |
| lset.json                             | 56   |             |
| ltrim.json                            | 55   |             |
| memory-doctor.json                    | 21   |             |
| memory-help.json                      | 23   |             |
| memory-malloc-stats.json              | 21   |             |
| memory-purge.json                     | 19   |             |
| memory-stats.json                     | 122  |             |
| memory-usage.json                     | 59   |             |
| memory.json                           | 10   |             |
| mget.json                             | 64   |             |
| migrate.json                          | 182  |             |
| module-help.json                      | 23   |             |
| module-list.json                      | 48   |             |
| module-load.json                      | 33   |             |
| module-loadex.json                    | 52   |             |
| module-unload.json                    | 27   |             |
| module.json                           | 10   |             |
| monitor.json                          | 17   |             |
| move.json                             | 62   |             |
| mset.json                             | 63   |             |
| msetnx.json                           | 68   |             |
| multi.json                            | 24   |             |
| object-encoding.json                  | 59   |             |
| object-freq.json                      | 51   |             |
| object-help.json                      | 26   |             |
| object-idletime.json                  | 51   |             |
| object-refcount.json                  | 51   |             |
| object.json                           | 10   |             |
| persist.json                          | 57   |             |
| pexpire.json                          | 95   |             |
| pexpireat.json                        | 95   |             |
| pexpiretime.json                      | 62   |             |
| pfadd.json                            | 64   |             |
| pfcount.json                          | 51   |             |
| pfdebug.json                          | 53   |             |
| pfmerge.json                          | 74   |             |
| pfselftest.json                       | 23   |             |
| ping.json                             | 41   |             |
| psetex.json                           | 61   |             |
| psubscribe.json                       | 25   |             |
| psync.json                            | 26   |             |
| pttl.json                             | 71   |             |
| publish.json                          | 34   |             |
| pubsub-channels.json                  | 32   |             |
| pubsub-help.json                      | 23   |             |
| pubsub-numpat.json                    | 22   |             |
| pubsub-numsub.json                    | 29   |             |
| pubsub-shardchannels.json             | 32   |             |
| pubsub-shardnumsub.json               | 29   |             |
| pubsub.json                           | 10   |             |
| punsubscribe.json                     | 26   |             |
| quit.json                             | 30   |             |
| randomkey.json                        | 35   |             |
| readonly.json                         | 22   |             |
| readwrite.json                        | 22   |             |
| rename.json                           | 73   |             |
| renamenx.json                         | 87   |             |
| replconf.json                         | 24   |             |
| replicaof.json                        | 60   |             |
| reset.json                            | 25   |             |
| restore-asking.json                   | 103  |             |
| restore.json                          | 99   |             |
| role.json                             | 135  |             |
| rpop.json                             | 77   |             |
| rpoplpush.json                        | 86   |             |
| rpush.json                            | 62   |             |
| rpushx.json                           | 62   |             |
| sadd.json                             | 61   |             |
| save.json                             | 20   |             |
| scan.json                             | 73   |             |
| scard.json                            | 49   |             |
| script-debug.json                     | 44   |             |
| script-exists.json                    | 45   |             |
| script-flush.json                     | 51   |             |
| script-help.json                      | 26   |             |
| script-kill.json                      | 26   |             |
| script-load.json                      | 33   |             |
| script.json                           | 10   |             |
| sdiff.json                            | 56   |             |
| sdiffstore.json                       | 74   |             |
| select.json                           | 28   |             |
| sentinel-ckquorum.json                | 27   |             |
| sentinel-config.json                  | 122  |             |
| sentinel-debug.json                   | 50   |             |
| sentinel-failover.json                | 26   |             |
| sentinel-flushconfig.json             | 21   |             |
| sentinel-get-master-addr-by-name.json | 39   |             |
| sentinel-help.json                    | 25   |             |
| sentinel-info-cache.json              | 65   |             |
| sentinel-is-master-down-by-addr.json  | 62   |             |
| sentinel-master.json                  | 30   |             |
| sentinel-masters.json                 | 27   |             |
| sentinel-monitor.json                 | 38   |             |
| sentinel-myid.json                    | 21   |             |
| sentinel-pending-scripts.json         | 53   |             |
| sentinel-remove.json                  | 26   |             |
| sentinel-replicas.json                | 33   |             |
| sentinel-reset.json                   | 27   |             |
| sentinel-sentinels.json               | 33   |             |
| sentinel-set.json                     | 41   |             |
| sentinel-simulate-failure.json        | 53   |             |
| sentinel-slaves.json                  | 38   |             |
| sentinel.json                         | 15   |             |
| set.json                              | 153  |             |
| setbit.json                           | 65   |             |
| setex.json                            | 61   |             |
| setnx.json                            | 67   |             |
| setrange.json                         | 58   |             |
| shutdown.json                         | 70   |             |
| sinter.json                           | 56   |             |
| sintercard.json                       | 61   |             |
| sinterstore.json                      | 74   |             |
| sismember.json                        | 60   |             |
| slaveof.json                          | 65   |             |
| slowlog-get.json                      | 75   |             |
| slowlog-help.json                     | 23   |             |
| slowlog-len.json                      | 27   |             |
| slowlog-reset.json                    | 24   |             |
| slowlog.json                          | 10   |             |
| smembers.json                         | 55   |             |
| smismember.json                       | 67   |             |
| smove.json                            | 85   |             |
| sort.json                             | 163  |             |
| sort_ro.json                          | 133  |             |
| spop.json                             | 81   |             |
| spublish.json                         | 52   |             |
| srandmember.json                      | 84   |             |
| srem.json                             | 61   |             |
| sscan.json                            | 82   |             |
| ssubscribe.json                       | 43   |             |
| strlen.json                           | 49   |             |
| subscribe.json                        | 26   |             |
| substr.json                           | 61   |             |
| sunion.json                           | 56   |             |
| sunionstore.json                      | 74   |             |
| sunsubscribe.json                     | 44   |             |
| swapdb.json                           | 32   |             |
| sync.json                             | 16   |             |
| time.json                             | 29   |             |
| touch.json                            | 54   |             |
| ttl.json                              | 71   |             |
| type.json                             | 56   |             |
| unlink.json                           | 55   |             |
| unsubscribe.json                      | 26   |             |
| unwatch.json                          | 24   |             |
| wait.json                             | 35   |             |
| waitaof.json                          | 53   |             |
| watch.json                            | 51   |             |
| xack.json                             | 59   |             |
| xadd.json                             | 162  |             |
| xautoclaim.json                       | 159  |             |
| xclaim.json                           | 139  |             |
| xdel.json                             | 55   |             |
| xgroup-create.json                    | 86   |             |
| xgroup-createconsumer.json            | 65   |             |
| xgroup-delconsumer.json               | 58   |             |
| xgroup-destroy.json                   | 60   |             |
| xgroup-help.json                      | 26   |             |
| xgroup-setid.json                     | 80   |             |
| xgroup.json                           | 10   |             |
| xinfo-consumers.json                  | 81   |             |
| xinfo-groups.json                     | 93   |             |
| xinfo-help.json                       | 26   |             |
| xinfo-stream.json                     | 362  |             |
| xinfo.json                            | 10   |             |
| xlen.json                             | 49   |             |
| xpending.json                         | 161  |             |
| xrange.json                           | 88   |             |
| xread.json                            | 109  |             |
| xreadgroup.json                       | 135  |             |
| xrevrange.json                        | 87   |             |
| xsetid.json                           | 73   |             |
| xtrim.json                            | 109  |             |
| zadd.json                             | 145  |             |
| zcard.json                            | 48   |             |
| zcount.json                           | 57   |             |
| zdiff.json                            | 86   |             |
| zdiffstore.json                       | 78   |             |
| zincrby.json                          | 59   |             |
| zinter.json                           | 116  |             |
| zintercard.json                       | 61   |             |
| zinterstore.json                      | 109  |             |
| zlexcount.json                        | 58   |             |
| zmpop.json                            | 112  |             |
| zmscore.json                          | 66   |             |
| zpopmax.json                          | 90   |             |
| zpopmin.json                          | 90   |             |
| zrandmember.json                      | 102  |             |
| zrange.json                           | 138  |             |
| zrangebylex.json                      | 81   |             |
| zrangebyscore.json                    | 120  |             |
| zrangestore.json                      | 119  |             |
| zrank.json                            | 87   |             |
| zrem.json                             | 61   |             |
| zremrangebylex.json                   | 56   |             |
| zremrangebyrank.json                  | 56   |             |
| zremrangebyscore.json                 | 56   |             |
| zrevrange.json                        | 95   |             |
| zrevrangebylex.json                   | 81   |             |
| zrevrangebyscore.json                 | 119  |             |
| zrevrank.json                         | 87   |             |
| zscan.json                            | 82   |             |
| zscore.json                           | 61   |             |
| zunion.json                           | 116  |             |
| zunionstore.json                      | 108  |             |


## the build script

```makefile
REDIS_SERVER_NAME=redis-server$(PROG_SUFFIX)
REDIS_SENTINEL_NAME=redis-sentinel$(PROG_SUFFIX)
REDIS_SERVER_OBJ=adlist.o quicklist.o ae.o anet.o dict.o server.o sds.o zmalloc.o lzf_c.o lzf_d.o pqsort.o zipmap.o sha1.o ziplist.o release.o networking.o util.o object.o db.o replication.o rdb.o t_string.o t_list.o t_set.o t_zset.o t_hash.o config.o aof.o pubsub.o multi.o debug.o sort.o intset.o syncio.o cluster.o crc16.o endianconv.o slowlog.o eval.o bio.o rio.o rand.o memtest.o syscheck.o crcspeed.o crc64.o bitops.o sentinel.o notify.o setproctitle.o blocked.o hyperloglog.o latency.o sparkline.o redis-check-rdb.o redis-check-aof.o geo.o lazyfree.o module.o evict.o expire.o geohash.o geohash_helper.o childinfo.o defrag.o siphash.o rax.o t_stream.o listpack.o localtime.o lolwut.o lolwut5.o lolwut6.o acl.o tracking.o connection.o tls.o sha256.o timeout.o setcpuaffinity.o monotonic.o mt19937-64.o resp_parser.o call_reply.o script_lua.o script.o functions.o function_lua.o commands.o
REDIS_CLI_NAME=redis-cli$(PROG_SUFFIX)
REDIS_CLI_OBJ=anet.o adlist.o dict.o redis-cli.o zmalloc.o release.o ae.o redisassert.o crcspeed.o crc64.o siphash.o crc16.o monotonic.o cli_common.o mt19937-64.o
REDIS_BENCHMARK_NAME=redis-benchmark$(PROG_SUFFIX)
REDIS_BENCHMARK_OBJ=ae.o anet.o redis-benchmark.o adlist.o dict.o zmalloc.o redisassert.o release.o crcspeed.o crc64.o siphash.o crc16.o monotonic.o cli_common.o mt19937-64.o
REDIS_CHECK_RDB_NAME=redis-check-rdb$(PROG_SUFFIX)
REDIS_CHECK_AOF_NAME=redis-check-aof$(PROG_SUFFIX)
ALL_SOURCES=$(sort $(patsubst %.o,%.c,$(REDIS_SERVER_OBJ) $(REDIS_CLI_OBJ) $(REDIS_BENCHMARK_OBJ)))

all: $(REDIS_SERVER_NAME) $(REDIS_SENTINEL_NAME) $(REDIS_CLI_NAME) $(REDIS_BENCHMARK_NAME) $(REDIS_CHECK_RDB_NAME) $(REDIS_CHECK_AOF_NAME)
	@echo ""
	@echo "Hint: It's a good idea to run 'make test' ;)"
	@echo ""
```

REDIS_SERVER_OBJ:


| Object            | Description                                                                                       |
| ----------------- | ------------------------------------------------------------------------------------------------- |
| adlist.o          |                                                                                                   |
| quicklist.o       |                                                                                                   |
| ae.o              | event loop                                                                                        |
| anet.o            | a library to use POSIX networking in a simpler way                                                |
| dict.o            | a non-blocking hash table, which rehash incrementally                                             |
| server.o          | the entry point of the Redis server: `main()`, `struct redisServer`, `struct client`              |
| sds.o             | string library [https://github.com/antirez/sds](https://github.com/antirez/sds)                   |
| zmalloc.o         |                                                                                                   |
| lzf_c.o           |                                                                                                   |
| lzf_d.o           |                                                                                                   |
| pqsort.o          |                                                                                                   |
| zipmap.o          |                                                                                                   |
| sha1.o            |                                                                                                   |
| ziplist.o         |                                                                                                   |
| release.o         |                                                                                                   |
| networking.o      | the I/O functions with clients, masters and replicas                                              |
| util.o            |                                                                                                   |
| object.o          | defining Redis object: `robj`                                                                     |
| db.o              | generic commands on keys, operation on dataset directly                                           |
| replication.o     | master and replica role of Redis                                                                  |
| rdb.o             | RDB persistence: create the snapshots on disk                                                     |
| t_string.o        | String data type                                                                                  |
| t_list.o          | List data type                                                                                    |
| t_set.o           | Set data type                                                                                     |
| t_zset.o          | ZSet data type                                                                                    |
| t_hash.o          | Hash data type                                                                                    |
| config.o          |                                                                                                   |
| aof.o             | AOF persistence: rewriten when the append only file gets too big                                  |
| pubsub.o          |                                                                                                   |
| multi.o           |                                                                                                   |
| debug.o           |                                                                                                   |
| sort.o            |                                                                                                   |
| intset.o          |                                                                                                   |
| syncio.o          |                                                                                                   |
| cluster.o         | the Redis Cluster. [https://redis.io/topics/cluster-spec/](https://redis.io/topics/cluster-spec/) |
| crc16.o           |                                                                                                   |
| endianconv.o      |                                                                                                   |
| slowlog.o         |                                                                                                   |
| eval.o            |                                                                                                   |
| bio.o             |                                                                                                   |
| rio.o             |                                                                                                   |
| rand.o            |                                                                                                   |
| memtest.o         |                                                                                                   |
| syscheck.o        |                                                                                                   |
| crcspeed.o        |                                                                                                   |
| crc64.o           |                                                                                                   |
| bitops.o          |                                                                                                   |
| sentinel.o        |                                                                                                   |
| notify.o          |                                                                                                   |
| setproctitle.o    |                                                                                                   |
| blocked.o         |                                                                                                   |
| hyperloglog.o     |                                                                                                   |
| latency.o         |                                                                                                   |
| sparkline.o       |                                                                                                   |
| redis-check-rdb.o |                                                                                                   |
| redis-check-aof.o |                                                                                                   |
| geo.o             |                                                                                                   |
| lazyfree.o        |                                                                                                   |
| module.o          |                                                                                                   |
| evict.o           |                                                                                                   |
| expire.o          |                                                                                                   |
| geohash.o         |                                                                                                   |
| geohash_helper.o  |                                                                                                   |
| childinfo.o       |                                                                                                   |
| defrag.o          |                                                                                                   |
| siphash.o         |                                                                                                   |
| rax.o             |                                                                                                   |
| t_stream.o        | Streams data type                                                                                 |
| listpack.o        |                                                                                                   |
| localtime.o       |                                                                                                   |
| lolwut.o          |                                                                                                   |
| lolwut5.o         |                                                                                                   |
| lolwut6.o         |                                                                                                   |
| acl.o             |                                                                                                   |
| tracking.o        |                                                                                                   |
| connection.o      |                                                                                                   |
| tls.o             |                                                                                                   |
| sha256.o          |                                                                                                   |
| timeout.o         |                                                                                                   |
| setcpuaffinity.o  |                                                                                                   |
| monotonic.o       |                                                                                                   |
| mt19937-64.o      |                                                                                                   |
| resp_parser.o     |                                                                                                   |
| call_reply.o      |                                                                                                   |
| script_lua.o      | execute Lua code                                                                                  |
| script.o          | integration scripts: commands execution, set replication/resp, ...                                |
| functions.o       | FUNCTION command                                                                                  |
| function_lua.o    | contains the Lua engine implementation                                                            |
| commands.o        | auto generated by `utils/generate-command-code.py`       


# Benchmark

- `redis-benchmark`
- [FlameGraph](../../DevOps/Observability/OS%20Performance.ipynb)

# Misc

## pika
- https://github.com/OpenAtomFoundation/pika

> Pika is a high-performance, large-capacity, multi-tenant, data-persistent elastic KV data storage system using RocksDB as the storage engine. It is fully compatible with the Redis protocol and supports its commonly used data structures, such as string/hash/list/zset/set/geo/hyperloglog/pubsub/bitmap/stream, etc. [Redis Interface](https://github.com/OpenAtomFoundation/pika/wiki/pika-%E6%94%AF%E6%8C%81%E7%9A%84redis%E6%8E%A5%E5%8F%A3%E5%8F%8A%E5%85%BC%E5%AE%B9%E6%83%85%E5%86%B5).